# This notebook uses sympy to derive nonlinear coefficients for terms in the CdV equations.
## Produced as an appendix to Josh Dorrington's PhD thesis, 2021.


In [50]:
from sympy import *
import pickle
import os
import numpy as np
import itertools as it

def isolate(rhs,index,term):
    
    return simplify(rhs[index].coeff(term))*term

def print_eq(rhs,index,mode_num=10):
    eq=0
    for i,j in it.product(range(mode_num),range(mode_num)):
        eq=eq+isolate(rhs,index,xs[i]*xs[j])
    return eq


## We start by defining the variables and basis functions

In [10]:
x_modes=[-1,0,1]
y_modes=[1,2]
D=6
y=Symbol("y")
b=Symbol("b",nonzero=True,positive=True,rational=True)

#Here we initialise the mode variables and the spatial variables
jy=var("j_y",integer=True,nonzero=True,positive=True)
ky=var("k_y",integer=True,nonzero=True,positive=True)
ly=var("l_y",integer=True,nonzero=True,positive=True)

jx=var("j_x",integer=True)
kx=var("k_x",integer=True)
lx=var("l_x",integer=True)

x=var("x")
y=var("y")

In [3]:
#Initialise three basis functionals for calculating nonlinear products
jmode=Piecewise((sqrt(2)*cos(jy*y/b),Eq(jx,0)),(sqrt(2)*exp(I*jx*x)*sin(jy*y/b),True))
kmode=Piecewise((sqrt(2)*cos(ky*y/b),Eq(kx,0)),(sqrt(2)*exp(I*kx*x)*sin(ky*y/b),True))
lmode=Piecewise((sqrt(2)*cos(ly*y/b),Eq(lx,0)),(sqrt(2)*exp(I*lx*x)*sin(ly*y/b),True))

## We define a mapping from this basis to the real valued basis we will want at the end

In [4]:
#These are the descriptors used in CdV79
mode_label={(0,1):"A",(0,2):"B",(1,1):"C",(-1,1):"D",(1,2):"E",(-1,2):"F"}

#These are the descriptors of the real modes
x1={(0,1):1/b}
x2={(-1,1):1/(sqrt(2)*b),(1,1):1/(sqrt(2)*b)}
x3={(-1,1):-I/(sqrt(2)*b),(1,1):I/(sqrt(2)*b)}
x4={(0,2):1/b}
x5={(-1,2):1/(sqrt(2)*b),(1,2):1/(sqrt(2)*b)}
x6={(-1,2):-I/(sqrt(2)*b),(1,2):I/(sqrt(2)*b)}

real_modes=[x1,x2,x3,x4,x5,x6]
laplacian_coeffs=[1/b**2,1+1/b**2,1+1/b**2,4/b**2,1+4/b**2,1+4/b**2]

## This is the integrand we will use to calculate the $c_{ijk}$

In [5]:
integrand=cancel(jmode*simplify((diff(kmode,x)*diff(lmode,y)-diff(kmode,y)*diff(lmode,x))))
normalisation_factor=4*pi**2

## Calculate all the $c_{ijk}$ values:

In [7]:
cs={}
n=0
for jxval,kxval,lxval in it.product(x_modes,x_modes,x_modes):
    for jyval,kyval,lyval in it.product(y_modes,y_modes,y_modes):
        #In this case the jacobian vanishes for sure
        if (kxval==0) & (lxval==0):
            result=0

        #In this case the x integral vanishes so we dont care either
        elif jxval+kxval+lxval!=0:
            result=0
        else:
            result=integrate(integrate(integrand.subs({jy:jyval,ky:kyval,ly:lyval,jx:jxval,kx:kxval,lx:lxval}),(x,0,2*pi)),(y,0,b*pi))
            
        if result!=0:
            cs[((jxval,jyval),(kxval,kyval),(lxval,lyval))]=result/normalisation_factor
        n+=1


## Use those to calculate the nonlinear terms in the equations:

In [8]:
nl_terms={}
for triad in cs:
    (jx,jy),(kx,ky),(lx,ly)=triad
    
    a_j=jx**2+(jy/b)**2
    a_l=lx**2+(ly/b)**2
    nl_terms[triad]=cs[triad]*(a_l)/(a_j)

## Transform to the real basis and combine the whole RHS of each equation together:

In [11]:
real_coeffs={}
for j,k,l in it.product(range(D),range(D),range(D)):
    xj=real_modes[j]
    xk=real_modes[k]
    xl=real_modes[l]
    
    mode_term=0
    for m1,m2,m3 in it.product(xj,xk,xl):
        if (m1!=m2)&(m2!=m3)&(m1!=m3):
            try:
                r=nl_terms[(m1,m2,m3)]
            except KeyError:
                r=0
                
            mode_term+=r*xj[m1]*xk[m2]*xl[m3]
            
    if mode_term!=0:    
        real_coeffs[(j+1,k+1,l+1)]=mode_term

In [23]:
xs=[var(f"x_{i}",real=True,commutative=True) for i in range(1,D+1)]

rhsides=[0 for i in range(D)]
for (j,k,l) in real_coeffs:
    rhsides[j-1]=rhsides[j-1]+real_coeffs[(j,k,l)]*xs[k-1]*xs[l-1]

# The final result:

In [36]:
print(f"x_1 nonlinear tendency:")
print_eq(rhsides,0,mode_num=D)

x_1 nonlinear tendency:


0

In [37]:
print(f"x_2 nonlinear tendency:")
print_eq(rhsides,1,mode_num=D)

x_2 nonlinear tendency:


-8*sqrt(2)*x_1*x_3/(3*pi*b*(b**2 + 1)) - 64*sqrt(2)*x_4*x_6/(15*pi*b*(b**2 + 1))

In [38]:
print(f"x_3 nonlinear tendency:")
print_eq(rhsides,2,mode_num=D)

x_3 nonlinear tendency:


8*sqrt(2)*x_1*x_2/(3*pi*b*(b**2 + 1)) + 64*sqrt(2)*x_4*x_5/(15*pi*b*(b**2 + 1))

In [39]:
print(f"x_4 nonlinear tendency:")
print_eq(rhsides,3,mode_num=D)

x_4 nonlinear tendency:


16*sqrt(2)*x_2*x_6/(5*pi*b**3) - 16*sqrt(2)*x_3*x_5/(5*pi*b**3)

In [40]:
print(f"x_5 nonlinear tendency:")
print_eq(rhsides,4,mode_num=D)

x_5 nonlinear tendency:


-32*sqrt(2)*x_1*x_6*(b**2 + 3)/(15*pi*b**3*(b**2 + 4)) + 64*sqrt(2)*x_3*x_4*(3 - b**2)/(15*pi*b**3*(b**2 + 4))

In [41]:
print(f"x_6 nonlinear tendency:")
print_eq(rhsides,5,mode_num=D)

x_6 nonlinear tendency:


32*sqrt(2)*x_1*x_5*(b**2 + 3)/(15*pi*b**3*(b**2 + 4)) + 64*sqrt(2)*x_2*x_4*(b**2 - 3)/(15*pi*b**3*(b**2 + 4))

## Orographic terms can be done similarly:

In [53]:
hs=[0,1/b,0,0,0,0,0,0,0,0]
gamma=Symbol("\gamma",positive=True,nonzero=True)

In [55]:
orog_terms={}
for triad in cs:
    (jx,jy),(kx,ky),(lx,ly)=triad
    
    a_j=jx**2+(jy/b)**2
    a_l=lx**2+(ly/b)**2
    orog_terms[triad]=cs[triad]*-gamma/(a_j)
    
real_orog_coeffs={}
for j,k,l in it.product(range(D),range(D),range(D)):
    xj=real_modes[j]
    xk=real_modes[k]
    xl=real_modes[l]
    
    mode_term=0
    for m1,m2,m3 in it.product(xj,xk,xl):
        if (m1!=m2)&(m2!=m3)&(m1!=m3):
            try:
                r=orog_terms[(m1,m2,m3)]
            except KeyError:
                r=0
                
            mode_term+=r*xj[m1]*xk[m2]*xl[m3]*hs[l]
            
    if mode_term!=0:    
        real_orog_coeffs[(j+1,k+1,l+1)]=mode_term
        


In [67]:
#entry h(i,j,k) represents a term in the ith tendency equation of the form h * x_j
real_orog_coeffs

{(1, 3, 2): 4*sqrt(2)*\gamma/(3*pi*b**2),
 (3, 1, 2): -4*sqrt(2)*\gamma/(3*pi*b**4*(1 + b**(-2))),
 (4, 6, 2): 8*sqrt(2)*\gamma/(15*pi*b**2),
 (6, 4, 2): -32*sqrt(2)*\gamma/(15*pi*b**4*(1 + 4/b**2))}

# Extending to 10 modes is as simple as redefining our basis functions:

In [ ]:
x_modes=[-2,-1,0,1,2]
y_modes=[1,2]
D=10
#These are the descriptors used in CdV79
mode_label={(0,1):"A",(0,2):"B",(1,1):"C",(-1,1):"D",(1,2):"E",(-1,2):"F",(-2,1):"G",(2,1):"H",(-2,2):"I",(2,2):"J"}

#These are the descriptors of the real modes
x1={(0,1):1}
x2={(-1,1):1/(sqrt(2)),(1,1):1/(sqrt(2))}
x3={(-1,1):-I/(sqrt(2)),(1,1):I/(sqrt(2))}
x4={(0,2):1}
x5={(-1,2):1/(sqrt(2)),(1,2):1/(sqrt(2))}
x6={(-1,2):-I/(sqrt(2)),(1,2):I/(sqrt(2))}

x7={(-2,1):1/(sqrt(2)),(2,1):1/(sqrt(2))}
x8={(-2,1):-I/(sqrt(2)),(2,1):I/(sqrt(2))}

x9={(-2,2):1/(sqrt(2)),(2,2):1/(sqrt(2))}
x10={(-2,2):-I/(sqrt(2)),(2,2):I/(sqrt(2))}

real_modes=[x1,x2,x3,x4,x5,x6,x7,x8,x9,x10]

cs={}
n=0
for jxval,kxval,lxval in it.product(x_modes,x_modes,x_modes):
    for jyval,kyval,lyval in it.product(y_modes,y_modes,y_modes):
        #In this case the jacobian vanishes for sure
        if (kxval==0) & (lxval==0):
            result=0

        #In this case the x integral vanishes so we dont care either
        elif jxval+kxval+lxval!=0:
            result=0
        else:
            result=integrate(integrate(integrand.subs({jy:jyval,ky:kyval,ly:lyval,jx:jxval,kx:kxval,lx:lxval}),(x,0,2*pi)),(y,0,b*pi))
            
        if result!=0:
            cs[((jxval,jyval),(kxval,kyval),(lxval,lyval))]=result/normalisation_factor
        n+=1
        
nl_terms={}
for triad in cs:
    (jx,jy),(kx,ky),(lx,ly)=triad
    
    a_j=jx**2+(jy/b)**2
    a_l=lx**2+(ly/b)**2
    nl_terms[triad]=cs[triad]*(a_l)/(a_j)
    
    
real_coeffs={}
for j,k,l in it.product(range(D),range(D),range(D)):
    xj=real_modes[j]
    xk=real_modes[k]
    xl=real_modes[l]
    
    mode_term=0
    for m1,m2,m3 in it.product(xj,xk,xl):
        if (m1!=m2)&(m2!=m3)&(m1!=m3):
            try:
                r=nl_terms[(m1,m2,m3)]
            except KeyError:
                r=0
                
            mode_term+=r*xj[m1]*xk[m2]*xl[m3]
            
    if mode_term!=0:    
        real_coeffs[(j+1,k+1,l+1)]=mode_term
        
xs=[var(f"x_{i}",real=True,commutative=True) for i in range(1,D+1)]

rhsides=[0 for i in range(D)]
for (j,k,l) in real_coeffs:
    rhsides[j-1]=rhsides[j-1]+real_coeffs[(j,k,l)]*xs[k-1]*xs[l-1]

In [13]:
print_eq(rhsides,0)

0

In [15]:
print_eq(rhsides,1)

-8*sqrt(2)*b**2*x_1*x_3/(3*pi*(b**2 + 1)) - 64*sqrt(2)*b**2*x_4*x_6/(15*pi*(b**2 + 1)) + 9*x_5*x_8*(1 - b**2)/(2*(b**2 + 1)) + 9*x_6*x_7*(b**2 - 1)/(2*(b**2 + 1))

In [16]:
print_eq(rhsides,2)

8*sqrt(2)*b**2*x_1*x_2/(3*pi*(b**2 + 1)) + 64*sqrt(2)*b**2*x_4*x_5/(15*pi*(b**2 + 1)) + 9*x_5*x_7*(b**2 - 1)/(2*(b**2 + 1)) + 9*x_6*x_8*(b**2 - 1)/(2*(b**2 + 1))

In [17]:
print_eq(rhsides,3)

32*sqrt(2)*x_10*x_7/(5*pi) + 16*sqrt(2)*x_2*x_6/(5*pi) - 16*sqrt(2)*x_3*x_5/(5*pi) - 32*sqrt(2)*x_8*x_9/(5*pi)

In [18]:
print_eq(rhsides,4)

9*b**2*x_2*x_8/(2*(b**2 + 4)) - 18*b**2*x_3*x_7/(4*b**2 + 16) - 32*sqrt(2)*x_1*x_6*(b**2 + 3)/(15*pi*(b**2 + 4)) + 64*sqrt(2)*x_3*x_4*(3 - b**2)/(15*pi*(b**2 + 4))

In [19]:
print_eq(rhsides,5)

-18*b**2*x_2*x_7/(4*b**2 + 16) - 18*b**2*x_3*x_8/(4*b**2 + 16) + 32*sqrt(2)*x_1*x_5*(b**2 + 3)/(15*pi*(b**2 + 4)) + 64*sqrt(2)*x_2*x_4*(b**2 - 3)/(15*pi*(b**2 + 4))

In [20]:
print_eq(rhsides,6)

-64*sqrt(2)*b**2*x_1*x_8/(3*pi*(4*b**2 + 1)) - 512*sqrt(2)*b**2*x_10*x_4/(15*pi*(4*b**2 + 1)) + 9*x_2*x_6/(2*(4*b**2 + 1)) + 9*x_3*x_5/(2*(4*b**2 + 1))

In [21]:
print_eq(rhsides,7)

64*sqrt(2)*b**2*x_1*x_7/(3*pi*(4*b**2 + 1)) + 512*sqrt(2)*b**2*x_4*x_9/(15*pi*(4*b**2 + 1)) - 18*x_2*x_5/(16*b**2 + 4) + 9*x_3*x_6/(2*(4*b**2 + 1))

In [22]:
print_eq(rhsides,8)

-16*sqrt(2)*x_1*x_10*(4*b**2 + 3)/(15*pi*(b**2 + 1)) + 32*sqrt(2)*x_4*x_8*(3 - 4*b**2)/(15*pi*(b**2 + 1))

In [23]:
print_eq(rhsides,9)

16*sqrt(2)*x_1*x_9*(4*b**2 + 3)/(15*pi*(b**2 + 1)) + 32*sqrt(2)*x_4*x_7*(4*b**2 - 3)/(15*pi*(b**2 + 1))

In [ ]:
orog_terms={}
for triad in cs:
    (jx,jy),(kx,ky),(lx,ly)=triad
    
    a_j=jx**2+(jy/b)**2
    a_l=lx**2+(ly/b)**2
    orog_terms[triad]=cs[triad]*-gamma/(a_j)
    
real_orog_coeffs={}
for j,k,l in it.product(range(D),range(D),range(D)):
    xj=real_modes[j]
    xk=real_modes[k]
    xl=real_modes[l]
    
    mode_term=0
    for m1,m2,m3 in it.product(xj,xk,xl):
        if (m1!=m2)&(m2!=m3)&(m1!=m3):
            try:
                r=orog_terms[(m1,m2,m3)]
            except KeyError:
                r=0
                
            mode_term+=r*xj[m1]*xk[m2]*xl[m3]*hs[l]
            
    if mode_term!=0:    
        real_orog_coeffs[(j+1,k+1,l+1)]=mode_term
#entry h(i,j,k) represents a term in the ith tendency equation of the form h * x_j
real_orog_coeffs 

# If you have any questions about this code please feel free to contact (hopefully by then Dr.) Josh Dorrington on joshua.dorrington@physics.ox.ac.uk or jcfdorrington@gmail.com